In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Ex from Park's notes
Implement Power Iteration with a 4x4 symmertic matrix, A, and an asymmetric one, B.
$\\ A = \begin{bmatrix} 1 & 2 & 3 & 4 \\ 4 & 5 & 6 & 7 \\ 2 & 1 & 5 & 0 \\ 4 & 2 & 1 & 0 \end{bmatrix}, B = \begin{bmatrix} 1 & 2 & 2 & 4 \\ 2 & 5 & 6 & 2 \\ 2 & 6 & 5 & 0 \\ 4 & 2 & 0 & 0 \end{bmatrix}$

In [2]:
A = np.array([  [1 , 2 , 3 , 4],
                [4 , 5 , 6 , 7],
                [2 , 1 , 5 , 0],
                [4 , 2 , 1 , 0]], dtype=np.float64)

B = np.array([  [1 , 2 , 2 , 4],
                [2 , 5 , 6 , 2],
                [2 , 6 , 5 , 0],
                [4 , 2 , 0 , 0]], dtype=np.float64)

In [4]:
def power_iteration(A, x0 = None, max_iter = 20):
    '''
    Returns an approximation of the eigenvector associated with the dominant
    eigenvalue of a matrix
    Input:
    A - matrix
    x0 - initial guess, vector
    max_iter - maximum iterations for the power iteration algorithm
    Output:
    u - associated dominant eigenvector (approximation)
    l - dominant eigenvalue (approximation)
    '''
    if not x0:
        x = np.zeros(A.shape[1])
        x[0] = 1.
    else:
        x = x0
    
    for j in range(max_iter):
        u = x / np.linalg.norm(x)
        x = A @ u
        l = np.dot(u,x)

    u = x / np.linalg.norm(x)

    return u, l

In [6]:
x1 = power_iteration(A)
print(x1)
x2 = power_iteration(B)
print(x2)

(array([0.3638539 , 0.84147511, 0.25701054, 0.30573773]), 11.105519741121565)
(array([0.30769905, 0.67185537, 0.64017353, 0.21002263]), 12.258235581068476)


### Hessenberg Computation

Basically similar to QR factorization via Householder reflector
* Use the same reflection operator once x is determined
* Slicing for matrix multiplication are apparent when writing out the procedure in a block form

Differences of HB form compared to QR fact.
* Only square matrix is allowed b/c of similarity to matrices
* loop over the first m-2 columns
* x (reflected vector) is sliced from one entry lower in row index than QR factorization
    * k (length of reflected vector) is smaller than QR by 1
* $\hat{H}$ is also multiplied on the right after on the left
* If $B = QAQ^T$ is desired, multiply Q by H on the left
    * If $A = QBQ^T$ is wanted, multiply Q by H on the right

In [5]:
def hessenberg1(A):
    '''
    Returns an upper Hessenberg form similar to the given matrix
    Input: A - square matrix
    Outputs: B, Q - Hessenberg matrix and an orthogonal matrix that
    satisfies B = Q A Q^T
    '''
    m, n = A.shape
    assert m == n
    B = A.copy()
    Q = np.eye(m)

    for i in range(m-2):
        k = m - i - 1

        x = B[(i+1):, i]
        w = np.zeros_like(x)
        w[0] = - np.sign(x[0]) * np.linalg.norm(x)

        v = w - x
        H = np.eye(k) - 2. * (np.outer(v, v)) / (np.dot(v,v))
        B[(i+1):, :] = H @ B[(i+1):, :]
        B[:, (i+1):] = B[:, (i+1):] @ H

        # multiply by H on the left: e.g. Q = H_3 H_2 H_1
        Q[(i+1):, :] = H @ Q[(i+1):, :]
    
    return B, Q

In [ ]:
# From Park's notes, checking function
m = 5
A = np.random.rand(m,m)
B, Q = hessenberg1(A)

with np.printoptions(precision=2, suppress=True):
    print(B)
    print(f"upper Hessenberg form? --> {np.allclose(np.tril(B, -2), np.zeros((m,m)))}")
    print(Q)
    print(Q.T @ Q)
    print(f"Q orthogonal? --> {np.allclose(Q.T @ Q, np.eye(m))}")
    print(f"B = QAQ^T? --> {np.allclose(B, Q @ A @ Q.T)}")

In [7]:
# We can make a better function for the HB form by
# making the code more efficient

def hessenberg(A):
    '''
    Returns an upper Hessenberg form similar to the given matrix
    Input: A - square matrix
    Outputs: B, Q - Hessenberg matrix and an orthogonal matrix that
    satisfies B = Q A Q^T
    '''
    m, n = A.shape
    assert m == n
    B = A.copy()
    Q = np.eye(m)

    for i in range(m-2):
        k = m - i - 1

        x = B[(i+1):, i].reshape(-1,1)
        w = np.zeros_like(x).reshape(-1,1)
        w[0] = - np.sign(x[0]) * np.linalg.norm(x)

        v = w - x
        v_ = ((2./(v.T @ v))*v)
        
        B[(i+1):, :] = B[(i+1):, :] - v_ @ (v.T @ B[(i+1):, :])
        B[:, (i+1):] = B[:, (i+1):] - (B[:, (i+1):] @ v_) @ v.T

        # Multiply by H on the left: e.g. Q = H_3 H_2 H_1 
        Q[(i+1):, :] = Q[(i+1):, :] - v_ @ (v.T @ Q[(i+1):, :])
        
    return B, Q

#### Example

Reduce the following (assymetric) matrix to upper HB form by means of unitary similarity transformas. And then carry out QR algorithm for 10 iterations.

$A = \begin{bmatrix} 1 & 2 & 3 & 4 \\ 4 & 5 & 6 & 7 \\ 2 & 1 & 5 & 0 \\ 4 & 2 & 1 & 0 \end{bmatrix}$

In [8]:
# Preliminary - unshifted QR algorithm
from QR import *

def qr_alg_unshift(A, max_iter=10):
    """
    Return approximate Schur factorization using QR algorithm.

    Input:
        A (array): A square matrix
    Output:
        T (array): approximate Schur form of A
        U (array): The unitary matrix involved in the similarity T=U^H*A*U
    Note: 
        - When A is a real matrix, T is an approxiate real Schur form.
        In this case, U is an orthogonal matrix.
        - 
    """
    m = A.shape[0]
    T = A.copy()
    U = np.eye(m)
    for _ in range(max_iter):
        Q, R = qr(T)
        T = R @ Q
        U = U @ Q
    
    return T, U

In [9]:
"""
Symmetric case:
    max_iter = 105
Asymmetric case:
    max_iter = 6
"""

# A = np.array([  [1 , 2 , 3 , 4],
#                 [4 , 5 , 6 , 7],
#                 [2 , 1 , 5 , 0],
#                 [4 , 2 , 1 , 0]], dtype=np.float64)

A = np.array([  [1 , 2 , 2 , 4],
                [2 , 5 , 6 , 2],
                [2 , 6 , 5 , 0],
                [4 , 2 , 0 , 0]], dtype=np.float64)

B, Q = hessenberg(A)
T, U = qr_alg_unshift(B, max_iter=100)

with np.printoptions(precision=5, suppress=True):
    print(B)
    print(T)
    print(np.allclose(T, U.T @ B @ U))
    print(np.allclose(B @ U, np.diag(T)*U))

[[ 1.      -4.89898 -0.      -0.     ]
 [-4.89898  5.       5.7735   0.     ]
 [-0.       5.7735   5.4     -2.12289]
 [-0.      -0.      -2.12289 -0.4    ]]
[[12.25824 -0.       0.      -0.     ]
 [-0.      -3.95885  0.00003  0.     ]
 [-0.       0.00003  3.52016 -0.     ]
 [-0.      -0.      -0.      -0.81954]]
True
False
